# Lithium Niobate


## Crystal Band Structure

In [ ]:
from photonic_crystal import CrystalSlab, Crystal2D
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from crystal_geometries import Crystal2D_Geometry, CrystalSlab_Geometry
from crystal_materials import Crystal_Materials
# Create an instance of the Crystal2D class
# Set the environmental variable before launching MPI processes
import os
os.environ["OMPI_MCA_opal_cuda_support"] = "true"
import meep as mp


# Define the materials of the crystal
material = Crystal_Materials()
material.atom = {"epsilon": 1}
material.bulk = {"epsilon_diag": mp.Vector3(4.88, 4.53, 4.88)}

# Define the geometry of the crystal
geometry = Crystal2D_Geometry(material=material, 
                              geometry_type='elliptical', 

                              a = 0.5,
                              b = 0.5)


# Define the simulation parameters
num_bands = 13
resolution = 64
interp = 15
periods = 3
lattice_type = 'triangular'
pickle_id = 'LiNbO3_2D'


In [ ]:

crystal_2d = Crystal2D(lattice_type=lattice_type, 
                        num_bands=num_bands, 
                        resolution=resolution, 
                        interp=interp, 
                        periods=periods, 
                        pickle_id=pickle_id,
                        geometry=geometry,
                        material=material,
                        k_point_max=0.5,
                        use_XY=True)


# Set the solver
crystal_2d.set_solver()
crystal_2d.run_simulation(runner="run_zeven")
crystal_2d.run_simulation(runner="run_zodd")

print("Dummy simulation run")

# Extract data
crystal_2d.extract_data(periods=1)
print("Data extracted")

# Plot epsilon interactively
print("Start plotting epsilon")
fig_eps = crystal_2d.plot_epsilon()
fig_eps.update_layout(
    autosize=False,
    width=700,
    height=700
)
print("Ready to show epsilon plot")
fig_eps.show()

# Plot bands interactively
print("Start plotting bands")
fig_bands = crystal_2d.plot_bands(polarization="zodd", title='Bands', color='blue')
fig_bands = crystal_2d.plot_bands(polarization="zeven", title='Bands', color='red', fig=fig_bands)
fig_bands.update_layout(
    autosize=False,
    width=700,
    height=700
)

w_light = []
for k in crystal_2d.k_points_interpolated:
    w_light.append(np.linalg.norm(k))

import plotly.graph_objects as go
fig_bands.add_trace(
    go.Scatter(
        y= [w for w in w_light],
        mode='lines',
        marker=dict(
            size=5,
            color='black'
        ),
        name='Light line'
    )
)




print("Ready to show bands plot")
fig_bands.show()



[Vector3<0.5, 0.0, 0.0>, Vector3<0.46875, 0.0, 0.0>, Vector3<0.4375, 0.0, 0.0>, Vector3<0.40625, 0.0, 0.0>, Vector3<0.375, 0.0, 0.0>, Vector3<0.34375, 0.0, 0.0>, Vector3<0.3125, 0.0, 0.0>, Vector3<0.28125, 0.0, 0.0>, Vector3<0.25, 0.0, 0.0>, Vector3<0.21875, 0.0, 0.0>, Vector3<0.1875, 0.0, 0.0>, Vector3<0.15625, 0.0, 0.0>, Vector3<0.125, 0.0, 0.0>, Vector3<0.09375, 0.0, 0.0>, Vector3<0.0625, 0.0, 0.0>, Vector3<0.03125, 0.0, 0.0>, Vector3<0.0, 0.0, 0.0>, Vector3<0.0, 0.03125, 0.0>, Vector3<0.0, 0.0625, 0.0>, Vector3<0.0, 0.09375, 0.0>, Vector3<0.0, 0.125, 0.0>, Vector3<0.0, 0.15625, 0.0>, Vector3<0.0, 0.1875, 0.0>, Vector3<0.0, 0.21875, 0.0>, Vector3<0.0, 0.25, 0.0>, Vector3<0.0, 0.28125, 0.0>, Vector3<0.0, 0.3125, 0.0>, Vector3<0.0, 0.34375, 0.0>, Vector3<0.0, 0.375, 0.0>, Vector3<0.0, 0.40625, 0.0>, Vector3<0.0, 0.4375, 0.0>, Vector3<0.0, 0.46875, 0.0>, Vector3<0.0, 0.5, 0.0>]
[Vector3<0.5, 0.0, 0.0>, Vector3<0.46875, 0.0, 0.0>, Vector3<0.4375, 0.0, 0.0>, Vector3<0.40625, 0.0, 0.0>, V

Start plotting bands
Ready to show bands plot


## Minimize the bandgap

Let's check if the algorithm to find the candidate modes works

In [7]:
target_freq = 0.5376
target_polarization = "zodd"
target_runner = "run_zodd"
candidate_modes = crystal_2d.find_candidate_modes(target_freq=target_freq, runner=target_runner, polarization=target_polarization)
print("Number of candidate modes:", len(candidate_modes))
print("Candidate modes frequencies:")
for mode in candidate_modes:
    print(" ",mode["freq"], mode["polarization"])

gaps = [candidate_modes[i+1]['freq'] - candidate_modes[i]['freq'] for i in range(len(candidate_modes)-1)]
print("Gaps between candidate modes frequencies:", [f"{gap:.4e}" for gap in gaps])

Number of candidate modes: 3
Candidate modes frequencies:
  0.5376035440795937 zodd
  0.5720898300258265 zodd
  0.5721219119039755 zodd
Gaps between candidate modes frequencies: ['3.4486e-02', '3.2082e-05']


test the partial geometry

In [8]:
print(crystal_2d.geometry.kwargs)
pg = crystal_2d.geometry.to_partial(["r"])
print(pg().kwargs)
kwargs = {"r": 0.1}
geometry = pg(**kwargs)
print(geometry.kwargs)

gaps = crystal_2d.find_gaps_between_candidate_modes(target_freq=target_freq, runner=target_runner, polarization=target_polarization, params={"r": 0.1})
print("r=0.1:", gaps)

gaps = crystal_2d.find_gaps_between_candidate_modes(target_freq=target_freq, runner=target_runner, polarization=target_polarization, params={"r": 0.35})
print("r=0.35:", gaps)

{'r': 0.35}
{}
{'r': 0.1}
r=0.1: [0.002515290054296404, 1.8729191285093982e-05]
r=0.35: [0.034486288115862296, 3.208181026603807e-05]


Now we want to minimize the gaps between these modes

In [12]:
parameters = {
    "r": 0.35,
}
bounds = {
    "r": (0.2, 0.5)
}

optimal_parameters = crystal_2d.find_dirac_point(target_freq=target_freq, 
                                                 runner=target_runner, 
                                                 polarization=target_polarization, 
                                                 params=parameters, 
                                                 bounds=bounds, 
                                                 verbose=True, 
                                                 maxiter=50)
print("Optimal parameters:", optimal_parameters)

Current parameters: {'r': '2.8215e-01'} Gaps: ['1.3515e-02', '9.8168e-06']
Current parameters: {'r': '2.7520e-01'} Gaps: ['1.2084e-02', '1.1501e-05']
Current parameters: {'r': '4.8139e-01'} Gaps: ['1.9689e-01', '2.4461e-04']
Current parameters: {'r': '3.8237e-01'} Gaps: ['5.0821e-02', '6.4912e-05']
Current parameters: {'r': '4.3737e-01'} Gaps: ['9.9569e-02', '1.4404e-04']
Current parameters: {'r': '2.1601e-01'} Gaps: ['3.8515e-03', '5.2123e-06']
Current parameters: {'r': '4.4184e-01'} Gaps: ['1.0581e-01', '1.2116e-04']
Current parameters: {'r': '2.2548e-01'} Gaps: ['4.7527e-03', '2.4105e-06']
Current parameters: {'r': '3.2639e-01'} Gaps: ['2.5534e-02', '3.1555e-05']
Current parameters: {'r': '4.1953e-01'} Gaps: ['7.9244e-02', '1.1030e-04']
Current parameters: {'r': '3.5459e-01'} Gaps: ['3.6471e-02', '4.5287e-05']
Current parameters: {'r': '3.0686e-01'} Gaps: ['1.9570e-02', '5.9032e-06']
Current parameters: {'r': '2.5728e-01'} Gaps: ['8.8858e-03', '1.1116e-05']
Current parameters: {'r':

SystemError: <built-in function mode_solver_solve_kpoint> returned a result with an exception set

We now use the optimal parameter to plot the band diagram

In [10]:
crystal_2d.geometry = crystal_2d.geometry.to_partial(["r"])(**optimal_parameters)


# Set the solver
crystal_2d.set_solver()
crystal_2d.run_simulation(runner="run_zeven")
crystal_2d.run_simulation(runner="run_zodd")

print("Dummy simulation run")

# Extract data
crystal_2d.extract_data(periods=1)
print("Data extracted")

# Plot epsilon interactively
print("Start plotting epsilon")
fig_eps = crystal_2d.plot_epsilon()
fig_eps.update_layout(
    autosize=False,
    width=700,
    height=700
)
print("Ready to show epsilon plot")
fig_eps.show()

# Plot bands interactively
print("Start plotting bands")
fig_bands = crystal_2d.plot_bands(polarization="zodd", title='Bands', color='blue')
fig_bands = crystal_2d.plot_bands(polarization="zeven", title='Bands', color='red', fig=fig_bands)
fig_bands.update_layout(
    autosize=False,
    width=700,
    height=700
)

w_light = []
for k in crystal_2d.k_points_interpolated:
    w_light.append(np.linalg.norm(k))

import plotly.graph_objects as go
fig_bands.add_trace(
    go.Scatter(
        y= [w for w in w_light],
        mode='lines',
        marker=dict(
            size=5,
            color='black'
        ),
        name='Light line'
    )
)




print("Ready to show bands plot")
fig_bands.show()



[Vector3<0.5, 0.0, 0.0>, Vector3<0.46875, 0.0, 0.0>, Vector3<0.4375, 0.0, 0.0>, Vector3<0.40625, 0.0, 0.0>, Vector3<0.375, 0.0, 0.0>, Vector3<0.34375, 0.0, 0.0>, Vector3<0.3125, 0.0, 0.0>, Vector3<0.28125, 0.0, 0.0>, Vector3<0.25, 0.0, 0.0>, Vector3<0.21875, 0.0, 0.0>, Vector3<0.1875, 0.0, 0.0>, Vector3<0.15625, 0.0, 0.0>, Vector3<0.125, 0.0, 0.0>, Vector3<0.09375, 0.0, 0.0>, Vector3<0.0625, 0.0, 0.0>, Vector3<0.03125, 0.0, 0.0>, Vector3<0.0, 0.0, 0.0>, Vector3<0.0, 0.03125, 0.0>, Vector3<0.0, 0.0625, 0.0>, Vector3<0.0, 0.09375, 0.0>, Vector3<0.0, 0.125, 0.0>, Vector3<0.0, 0.15625, 0.0>, Vector3<0.0, 0.1875, 0.0>, Vector3<0.0, 0.21875, 0.0>, Vector3<0.0, 0.25, 0.0>, Vector3<0.0, 0.28125, 0.0>, Vector3<0.0, 0.3125, 0.0>, Vector3<0.0, 0.34375, 0.0>, Vector3<0.0, 0.375, 0.0>, Vector3<0.0, 0.40625, 0.0>, Vector3<0.0, 0.4375, 0.0>, Vector3<0.0, 0.46875, 0.0>, Vector3<0.0, 0.5, 0.0>]
[Vector3<0.5, 0.0, 0.0>, Vector3<0.46875, 0.0, 0.0>, Vector3<0.4375, 0.0, 0.0>, Vector3<0.40625, 0.0, 0.0>, V

Start plotting bands
Ready to show bands plot
